# Peppol Readiness Checker

Upload your file with Name, VAT, and Country to check Peppol registration and VAT validity.

In [ ]:
# Install necessary libraries (skip if already installed)
!pip install pandas numpy requests dnspython ipywidgets fuzzywuzzy python-Levenshtein zeep

In [ ]:
# Upload file
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        df = pd.read_csv(io.BytesIO(uploaded[filename]))
    else:
        df = pd.read_excel(io.BytesIO(uploaded[filename]))
df.head()

In [ ]:
# Peppol + VAT functions
import requests
from zeep import Client
from fuzzywuzzy import process

def generate_peppol_participant_id(vat_number):
    return f"0088:{vat_number}".lower()

def check_peppol_dns(peppol_id):
    import dns.resolver
    try:
        domain = f"{peppol_id.replace(':','.')}.sml.peppolcentral.org"
        answers = dns.resolver.resolve(domain, 'A')
        return True, domain
    except:
        return False, None

def validate_vat_vies(country_code, vat_number):
    try:
        wsdl = "http://ec.europa.eu/taxation_customs/vies/checkVatService.wsdl"
        client = Client(wsdl=wsdl)
        result = client.service.checkVat(countryCode=country_code, vatNumber=vat_number)
        return result['valid']
    except:
        return "Error"

In [ ]:
# Run checks
results = []

for index, row in df.iterrows():
    name = str(row['Name'])
    vat = str(row['VAT']) if pd.notnull(row['VAT']) else None
    country = str(row['Country'])

    peppol_status = "No"
    peppol_id = None
    vat_status = "Not checked"

    if vat and country:
        peppol_id = generate_peppol_participant_id(vat)
        reg, domain = check_peppol_dns(peppol_id)
        peppol_status = "Yes" if reg else "No"
        vat_status = validate_vat_vies(country[:2].upper(), vat)
    elif name:
        if "test" in name.lower():
            peppol_status = "Likely"
            peppol_id = "Estimated by Name"
        else:
            peppol_status = "Unknown"
            peppol_id = "N/A"

    results.append({
        "Name": name,
        "VAT": vat,
        "Country": country,
        "Peppol Registered": peppol_status,
        "Peppol ID": peppol_id,
        "VAT Valid": vat_status
    })

results_df = pd.DataFrame(results)
results_df